# Balancer v2 Stable Pool Trade Functions

Information taken from
https://github.com/balancer-labs/balancer-v2-monorepo/blob/master/pkg/pool-stable/contracts/StableMath.sol

Python functions will be implemented in future

In [1]:
#imports

import numpy as np

### calculate-Invariant
This function calculates current pool invariant based on Curve's invariant formula

$$
An^n \sum{x_i} + D = ADn^n + \frac{D^{n+1}}{n^n\prod{x_i}},
$$

where $A$ is an amplification coefficient, found as follows:

$$
\chi = \frac{A\prod{x_i}}{(D/n)^n},
$$

$\chi$ is a dynamic leverage.

According to Curve, when $\chi \rightarrow \infty$, the formula tends to constant-sum invariant, and when $\chi = 0$, it represents constant-product invariant. Here A is a balance parameter of portfolio.

Rewriting in terms of A, we receive the formula that is used in StableMath for Balancer:


* $D$ - invariant

* $A$ - amp. coefficient

* $S$ - sum of balances

* $P$ - product of balances

* $n$ - number of tokens

$$
An^n\sum{x_i} + D = ADn^n + \frac{D^{n+1}}{n^n\prod{x_i}}
$$

In Balancer, this equation is solved for D via Newton-Raphson method

### get-Token-Balance-Given-Invariant-and-All-Other-Token-Balances
Calculates the balance of a token $t$ given the invariant and all other tokens balances

This is also solved using Newton-Raphson method.
Math is to be added.

### calc-Out-Given-In
Calculates how many tokens are retreived from pool given amount of tokens inserted in pool.

First, we calculate current invariant $D$ given $A$ and balances of tokens using **calculate-Invariant**

Final formula for amount of tokens $y$ retreived from the pool given amount $x$ of tokens injected into the pool:

$$
y^2 + (\sum\limits_{t\neq y}{x_t} - \frac{D}{An^n} - D)y - \frac{D^{n+1}}{An^{2n}P} = 0
$$

### calc-In-Given-Out
Calculates how many tokens ($x$) we need to inject into the pool given the amount of tokens we want to retreive from the pool.


$$
x^2 + (\sum\limits_{t\neq x}{x_t}-\frac{D}{An^n} - D)x - \frac{D^{n+1}}{An^{2n}P} = 0
$$

### calc-BPT-Out-Given-Exact-Tokens-In
Calculates amount of BP tokens minted given amount of tokens $a_t$ injected into the pool

Firstly, sum of balances $b_t$ is calculated:

$$
S = \sum\limits_{t}{b_t}
$$

Weight of balance of token $t$ is found as follows:

$$
w_t = \frac{b_t}{S}
$$

Secondly, weighted balance ratio and invariant ratio are calculated without concidering fees:

$$
BR_t = \frac{b_t + a_t}{b_t};\\
IR = \sum\limits_{t}{BR_t\cdot{w_t}}
$$

$IR$ shows how much the invariant has increased relatively to the old invariant if we concider no fees on the amount of the token injected into the pool.

Then, real amounts of token *in* ($a_t^i$) are calculated, taking into account the fee on the percentage excess. (As if the trader injected $a_t^i$ tokens without fees).

A check is performed if the balance ratio of a token is greater than the ideal ratio to decide whether to charge fees or not:

$$
BR_t > IR:\\
n_t = b_t\cdot{(IR-1)}, \hspace{10mm} t_t = a_t^i-n_t^i\\
a_t^i = n_t+(t_t\cdot(1-swapfee))\\
BR_t < IR:\\
a_t^i = a_t
$$

Here, $n_t$ is non-taxable amount in, $t_t$ - taxable amount in, $a_t^i$ - real amount in concidering fees, $a_t$ - initial amount in without concidering fees. Underscore $_t$ stands for token $t$.

After that balances are recalculated:

$$
b_t^{new} = b_t + a_t^i
$$

Then, current and new invariants are calculated using the **calculate-Invariant** function, based on old and new balances, respectively, and invariant ratio:

$$
IR = \frac{D^{new}}{D}
$$

And if $IR < 1$ (e.g. invariant $D$ didn't increase), no BP tokens are minted and return is 0. Else, BP tokens are minted:

$$
m = BPT_{TotalSupply}\cdot{(IR - 1)}
$$

### calc-Token-In-Given-Exact-BPT-Out
Calculates amount of token $t$ needed to be injected into the pool to receive back exact given amount of BP tokens $o$.

Firstly, old and new invariants are calculated based on old BPT balance and new:

$$
D^{new} = \frac{(BPT_{TotalSupply} + o)}{BPT_{TotalSupply}}\cdot{D}
$$

Secondly, we calculate amount of token $t$ ijected into the pool wihout fees simply by recalculating balances with new invariant using **get-Token-Balance-Given-Invariant-and-All-Other-Token-Balances**

$$
a_t = b_t^{new} - b_t
$$

Then, we calculate the sum of old balances to determine current weights of each token:

$$
S = \sum\limits_{t}{b_t}\\
w_t = \frac{b_t}{S}
$$


After that, we calculate amount of taxable percentage, taxable amount and non-taxable amount of token $t$:

$$
\% = 1 - w_t;\\
t_t = a_t\cdot{\%};\\
n_t = a_t - t_t.
$$

So, for $o$ tokens to be minted, we need to inject

$$
n_t + \frac{t_t}{1 - swapfee}
$$

of token $t$.